In [ ]:
# Task parser
from datetime import datetime
import csv

file_path = r'data_t1/data_for_spb_hakaton_entities1-Table 1.csv'

with open(file_path, newline='', encoding='utf-8') as csvfile:
    csv_reader = csv.reader(csvfile, delimiter=';', quotechar='"')
    task_data = [row for row in csv_reader]

print(task_data)

In [ ]:
# History parser

def add_or_update_key(dictionary : dict, key, value_to_add):
    dictionary[key] = dictionary.setdefault(key, [])
    dictionary[key].append(value_to_add)

file_path2 = r'data_t1/history-Table 1.csv'

with open(file_path2, newline='', encoding='utf-8') as csvfile:
    csv_reader2 = csv.reader(csvfile, delimiter=';', quotechar='"')
    history_data = [row for row in csv_reader2]

history_main_dict = {}

for pack in history_data:
    add_or_update_key(history_main_dict, pack[0], pack[1:])
    
print(history_main_dict['4337639'])

In [ ]:
# sprints-table parser



file_path2 = r'data_t1/sprints-Table 1.csv'

with open(file_path2, newline='', encoding='utf-8') as csvfile:
    csv_reader2 = csv.reader(csvfile, delimiter=';', quotechar='"')
    sprint_data = [row for row in csv_reader2]

for i in sprint_data:
    print(i)

In [ ]:
# investigate one sprint ! Спринт 2024.3.1.NPP Shared Sprint

def search_through_status_changes(key, fake_status_changes):
    status_change_trace = []
    for event in history_main_dict[key]:
        #! print(delta_time := ((datetime.strptime(sprint_end_date, '%Y-%m-%d %H:%M:%S.%f') - datetime.strptime(event[1], '%m/%d/%y %H:%M')).seconds / 3600))
        if event[0] == "Статус":
            status_change_trace.append(datetime.strptime(event[1], '%m/%d/%y %H:%M'))
    status_change_trace.sort()
    time_diference = []
    for i in range(1, len(status_change_trace)):
        time_diference.append((status_change_trace[i] - status_change_trace[i - 1]).total_seconds() / 60)
    if len(time_diference) != 0 and (sum(time_diference) / len(time_diference)) <= 5:
        fake_status_changes += 1
        
    print(key, time_diference, status_change_trace)
    return fake_status_changes
        
def search_for_last_day_status_change(key, sprint_end_date, last_day_completed_counter):
    for event in history_main_dict[key]:
        if (event[0] == "Статус") and (event[4].split(' ')[-1] == 'closed'):
           delta_time = ((datetime.strptime(sprint_end_date, '%Y-%m-%d %H:%M:%S.%f') - datetime.strptime(event[1], '%m/%d/%y %H:%M'))).days
           if delta_time == 0:
               print(key, event)
               last_day_completed_counter += 1
    return last_day_completed_counter

for pack in sprint_data:
    if 'Спринт 2024.3.2.NPP Shared Sprint' in pack:
        sprint_end_date = pack[3]
        enteties = pack[-1]
        enteties = enteties.replace('{', '')
        enteties = enteties.replace('}', '')
        entitiences = enteties.split(",")
        # print(entitiences)
        # print(sprint_end_date)
        break

fake_changes = 0
last_day_completions = 0
for instance in entitiences:
    fake_changes = search_through_status_changes(instance, fake_changes)
    last_day_completions = search_for_last_day_status_change(instance, sprint_end_date, last_day_completions)

print(fake_changes, fake_changes_percent := round((fake_changes / len(entitiences)) * 100) )
print(last_day_completions, last_day_completions_percent := round((last_day_completions / len(entitiences)) * 100) )






In [ ]:
date_string = '7/16/24 8:21'
date_string2 = '2024-07-16 19:00:00.000000'
parsed_date = datetime.strptime(date_string, '%m/%d/%y %H:%M')
parsed_date2 = datetime.strptime(date_string2, '%Y-%m-%d %H:%M:%S.%f')
x = [parsed_date2, parsed_date]
x.sort()
print(x)

In [ ]:
sprint_health_dict = {'irrelevant_status_track_tasks': f"{fake_changes_percent}%",
                      'last_day_completed_status_tasks': f"{last_day_completions_percent}%"
                   }

print(sprint_health_dict)